In [155]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv('file_out2.csv')  # Load your dataset
df.drop(["Unnamed: 0","TotalSales","Discount","CustomerID"], axis=1, inplace = True)
df['Date']=pd.to_datetime(df['Date'])
df=df.drop( df.query(" `Quantity`==0 ").index)
df


,InvoiceID,Date,ProductID,Quantity
0,328,2019-12-27,1684,4
1,329,2019-12-27,524,2
2,330,2019-12-27,192,4
3,330,2019-12-27,218,1
4,330,2019-12-27,247,4
...,...,...,...,...
29098,11092,2023-01-13,1644,1
29099,11093,2023-01-13,352,4
29100,11094,2023-01-13,683,4
29101,11095,2023-01-14,1830,4


#### Daily Sales Resampling

In [156]:
df = df.set_index('Date')
daily_sales = df['Quantity'].resample('D').sum()  # Resampling to daily totals
daily_sales_df=pd.DataFrame({'Date':daily_sales.index,'Quantity':daily_sales.values})


#### Time Features

In [157]:
daily_sales_df['Day_of_Week'] = daily_sales_df['Date'].dt.dayofweek
daily_sales_df['Day_of_Year'] = daily_sales_df['Date'].dt.dayofyear
daily_sales_df['Month'] = daily_sales_df['Date'].dt.month
daily_sales_df['Year'] = daily_sales_df['Date'].dt.year
daily_sales_df['Day_of_Month'] = daily_sales_df['Date'].dt.day
daily_sales_df['Week_Number'] = daily_sales_df['Date'].dt.isocalendar().week


In [158]:
daily_sales_df


,Date,Quantity,Day_of_Week,Day_of_Year,Month,Year,Day_of_Month,Week_Number
0,2019-01-02,109,2,2,1,2019,2,1
1,2019-01-03,51,3,3,1,2019,3,1
2,2019-01-04,65,4,4,1,2019,4,1
3,2019-01-05,52,5,5,1,2019,5,1
4,2019-01-06,0,6,6,1,2019,6,1
...,...,...,...,...,...,...,...,...
1539,2023-03-21,85,1,80,3,2023,21,12
1540,2023-03-22,137,2,81,3,2023,22,12
1541,2023-03-23,58,3,82,3,2023,23,12
1542,2023-03-24,48,4,83,3,2023,24,12


#### weekly sales

In [159]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

weekly_orders = daily_sales_df.groupby('Week_Number')['Quantity'].sum().reset_index()


fig = px.line(weekly_orders, x='Week_Number', y='Quantity', 
            labels={'Date': 'Date', 'Quantity': 'Total Orders'},
            title='Timeline of Orders')
fig.update_xaxes(tickangle=45) 
fig.show()

In [160]:
fig = px.box(weekly_orders, y='Quantity', title='Weekly Sales Boxplot')
fig.update_layout(yaxis_title='Sales')
fig.update_layout(xaxis_title='Week')
fig.show()

#### day of the week

In [161]:
day_of_week = daily_sales_df.groupby('Day_of_Week')['Quantity'].sum().reset_index()
fig = go.Figure([go.Bar(x=day_of_week['Day_of_Week'], y=day_of_week['Quantity'])])

fig.update_layout(
    title_text='Order volume by weekday',
    xaxis_title='day_of_week',
    yaxis_title='Quantity',
)

fig.show()

In [10]:
timeline_orders=df.groupby('Date')['Quantity'].sum().reset_index()
timeline_orders['Date'] = pd.to_datetime(timeline_orders['Date'])
timeline_orders = timeline_orders.sort_values(by='Date')

fig = px.line(timeline_orders, x='Date', y='Quantity', 
            labels={'Date': 'Date', 'Quantity': 'Total Orders'},
            title='Timeline of Orders')
fig.update_xaxes(tickangle=45) 
fig.show()

#### Grid search

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9]
}

grid_search = GridSearchCV(estimator=XGBRegressor(objective='reg:squarederror', random_state=42),
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_params


Fitting 3 folds for each of 72 candidates, totalling 216 fits


#### Oreders Count Prediction

In [18]:
daily_orders_count=df.groupby('Date')['InvoiceID'].count()
daily_orders_count=pd.DataFrame({'Date':daily_orders_count.index,'Orders_Count':daily_orders_count.values})


In [19]:
daily_orders_count.fillna(0,inplace=True)
daily_orders_count

,Date,Orders_Count
0,2019-01-02,30
1,2019-01-03,18
2,2019-01-04,14
3,2019-01-05,12
4,2019-01-07,25
...,...,...
1263,2023-03-21,22
1264,2023-03-22,17
1265,2023-03-23,12
1266,2023-03-24,10


#### Features Engineering for orders count

In [20]:
daily_orders_count['Day_of_Week'] = daily_orders_count['Date'].dt.dayofweek
daily_orders_count['Day_of_Year'] = daily_orders_count['Date'].dt.dayofyear
daily_orders_count['Month'] = daily_orders_count['Date'].dt.month
daily_orders_count['Year'] = daily_orders_count['Date'].dt.year
daily_orders_count['Week_Number'] = daily_orders_count['Date'].dt.isocalendar().week
daily_orders_count['Day_of_Month'] = daily_orders_count['Date'].dt.day
daily_orders_count['Prev_Day_Orders'] = daily_orders_count['Orders_Count'].shift(1)
daily_orders_count['Prev_Week_Orders'] = daily_orders_count['Orders_Count'].shift(7)
daily_orders_count['Prev_Month_Orders'] = daily_orders_count['Orders_Count'].shift(30)
# Rolling Statistics
window_size = 7  # Define the window size for rolling statistics
daily_orders_count['Rolling_Avg_Quantity'] = daily_orders_count['Orders_Count'].rolling(window=window_size).mean()
daily_orders_count['last_week_sum'] = daily_orders_count.set_index('Date').rolling(window='7D')['Orders_Count'].sum().reset_index(drop=True)


In [84]:
daily_orders_count.fillna(0,inplace=True)
daily_orders_count

,Date,Orders_Count,Day_of_Week,Day_of_Year,Month,Year,Week_Number,Day_of_Month,Prev_Day_Orders,Prev_Week_Orders,Prev_Month_Orders,Rolling_Avg_Quantity,last_week_sum
0,2019-01-02,30,2,2,1,2019,1,2,0.0,0.0,0.0,0.000000,30.0
1,2019-01-03,18,3,3,1,2019,1,3,30.0,0.0,0.0,0.000000,48.0
2,2019-01-04,14,4,4,1,2019,1,4,18.0,0.0,0.0,0.000000,62.0
3,2019-01-05,12,5,5,1,2019,1,5,14.0,0.0,0.0,0.000000,74.0
4,2019-01-07,25,0,7,1,2019,2,7,12.0,0.0,0.0,0.000000,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,2023-03-21,22,1,80,3,2023,12,21,27.0,24.0,17.0,19.142857,116.0
1264,2023-03-22,17,2,81,3,2023,12,22,22.0,18.0,15.0,19.000000,122.0
1265,2023-03-23,12,3,82,3,2023,12,23,17.0,11.0,14.0,19.142857,123.0
1266,2023-03-24,10,4,83,3,2023,12,24,12.0,11.0,13.0,19.000000,113.0


#### Splitting data train/test

In [ ]:
X = daily_orders_count.drop(['Date', 'Orders_Count'], axis=1) # features 
y = daily_orders_count['Orders_Count'] # target

Xd_train, Xd_test, yd_train, yd_test = train_test_split(X, y, test_size=0.1, random_state=42)


#### Build Model

In [123]:

daily_count_model =XGBRegressor( objective='reg:squarederror',
    learning_rate=0.1,
    max_depth=3,
    n_estimators=50,
    subsample=0.8,
    colsample_bytree=0.9)
daily_count_model.fit(Xd_train, yd_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

#### Prediction and Evaluation

In [124]:
predictionsD = daily_count_model.predict(Xd_test)

rmse = mean_squared_error(yd_test, predictionsD, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 8.882871027905686


In [135]:
trace_actual = go.Scatter(
    x=list(range(len(yd_test))),
    y=yd_test,
    mode='lines',
    name='Actual',
    line=dict(color='blue')
)

trace_predicted = go.Scatter(
    x=list(range(len(predictionsD))),
    y=predictionsD,
    mode='lines',
    name='Predicted',
    line=dict(color='red')
)

layout = go.Layout(
    title='Actual vs. Predicted Orders Count',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Orders Count'),
    showlegend=True,
)
fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)
fig.show()


#### Item_Level Sales Prediction

In [48]:
df1=pd.read_csv('file_out2.csv')
df1.drop('Unnamed: 0',axis=1,inplace=True)
df1['Date']=pd.to_datetime(df1['Date'])

In [71]:
# Grouping data by 'ProductID' and 'Date' to sum the 'Quantity' sold
grouped_data = df1.groupby(['ProductID', df1['Date'].dt.to_period('M')])['Quantity'].sum().reset_index()

grouped_data.rename(columns={'Date': 'Month'}, inplace=True)

# Create a DataFrame with all combinations of 'ProductID' and 'Month'
product_ids = df1['ProductID'].unique()
months = pd.period_range(df1['Date'].min().to_period('M'), df1['Date'].max().to_period('M'), freq='M')

all_combinations = []
for product_id in product_ids:
    for month in months:
        all_combinations.append({'ProductID': product_id, 'Month': month})

all_combinations_df = pd.DataFrame(all_combinations)

items_sales = pd.merge(all_combinations_df, grouped_data, on=['ProductID', 'Month'], how='left').fillna(0)
print( items_sales.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98940 entries, 0 to 98939
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype    
---  ------     --------------  -----    
 0   ProductID  98940 non-null  int64    
 1   Month      98940 non-null  period[M]
 2   Quantity   98940 non-null  float64  
dtypes: float64(1), int64(1), period[M](1)
memory usage: 2.3 MB
None


#### Features Engineering

In [74]:
items_sales['month'] = items_sales['Month'].dt.month
items_sales['Year'] = items_sales['Month'].dt.year
items_sales['Quarter'] = items_sales['Month'].dt.quarter
#Lag Features
max_lag = 2  #lag features for the last 2 months

for lag in range(1, max_lag + 1):
    items_sales[f'Quantity_Lag{lag}'] = items_sales.groupby('ProductID')['Quantity'].shift(lag).fillna(0)


In [75]:
items_sales

,ProductID,Month,Quantity,month,Year,Quantity_Lag1,Quantity_Lag2,Quarter
0,1684,2019-01,81.0,1,2019,0.0,0.0,1
1,1684,2019-02,4.0,2,2019,81.0,0.0,1
2,1684,2019-03,0.0,3,2019,4.0,81.0,1
3,1684,2019-04,0.0,4,2019,0.0,4.0,2
4,1684,2019-05,0.0,5,2019,0.0,0.0,2
...,...,...,...,...,...,...,...,...
98935,371,2022-11,0.0,11,2022,0.0,0.0,4
98936,371,2022-12,0.0,12,2022,0.0,0.0,4
98937,371,2023-01,8.0,1,2023,0.0,0.0,1
98938,371,2023-02,0.0,2,2023,8.0,0.0,1


#### Encoding Categorical Variables:

In [76]:
items_sales = pd.get_dummies(items_sales, columns=['ProductID'], prefix='Product')
items_sales


,Month,Quantity,month,Year,Quantity_Lag1,Quantity_Lag2,Quarter,Product_0,Product_1,Product_2,...,Product_1930,Product_1931,Product_1932,Product_1933,Product_1934,Product_1935,Product_1936,Product_1937,Product_1938,Product_1939
0,2019-01,81.0,1,2019,0.0,0.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2019-02,4.0,2,2019,81.0,0.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2019-03,0.0,3,2019,4.0,81.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2019-04,0.0,4,2019,0.0,4.0,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2019-05,0.0,5,2019,0.0,0.0,2,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98935,2022-11,0.0,11,2022,0.0,0.0,4,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98936,2022-12,0.0,12,2022,0.0,0.0,4,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98937,2023-01,8.0,1,2023,0.0,0.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98938,2023-02,0.0,2,2023,8.0,0.0,1,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### Dataset Splitting

In [93]:
from sklearn.model_selection import train_test_split

features = items_sales.drop(['Month', 'Quantity'], axis=1) # features 
target = items_sales['Quantity']  
Xi_train, Xi_test, yi_train, yi_test = train_test_split(features, target, test_size=0.2, shuffle=False)



#### Model Training and Evaluation

In [94]:
modelItems =XGBRegressor( objective='reg:squarederror',
    learning_rate=0.1,
    max_depth=3,
    n_estimators=50,
    subsample=0.8,
    colsample_bytree=0.9)

modelItems.fit(Xi_train, yi_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [114]:
predictionsI = modelItems.predict(Xi_test)

rmse = mean_squared_error(yi_test, predictionsI, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")


Root Mean Squared Error (RMSE): 3.1082899447560273


#### Features importances

In [108]:
feature_importance = daily_count_model.feature_importances_
booster = daily_count_model.get_booster()
feature_names = booster.feature_names
importance_dict = booster.get_score()
sorted_importances = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

feature_names = [x[0] for x in sorted_importances]
feature_importance = [x[1] for x in sorted_importances]

fig = go.Figure(go.Bar(
    x=feature_importance,
    y=feature_names,
    orientation='h'
))

fig.update_layout(
    title='Feature Importances',
    xaxis_title='Importance Score',
    yaxis_title='Feature Names',
    yaxis=dict(autorange="reversed"),  # Invert y-axis to display features from top to bottom
)

fig.show()